In [14]:
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib import request
from tqdm import tqdm
from os import path
import time
import re

### Helper Function

In [ ]:
def scrol_infinite(driver, scrool_pause_time = 1):

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(scrool_pause_time)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        
def download_all_videos(extracted_links, heading_list):
    """
    Download all videos and save in data dir
    """
    for i in tqdm(range(len(extracted_links))):
        request.urlretrieve(extracted_links[i], path.join("../data", heading_list[i]))

    return True

### Get list of Presentations

In [3]:
driver = webdriver.Chrome()
driver.get("https://slideslive.com/neurips")
scrol_infinite(driver)
html = driver.page_source

In [4]:
soup = BeautifulSoup(html)
presentations_div = soup.find('div',attrs={'class':'presentationsGrid'})
list_presentations_pages = []
list_presentations = presentations_div.find_all('div', attrs={'data-item-type':'presentation'})
for presentation in list_presentations:
    url = presentation.find('a')['href']
    list_presentations_pages.append(url)
    
print (len(list_presentations_pages))

269


#### Save presentations list to disk

In [5]:
with open('../data/all_urls.txt', 'w') as f:
    for item in list_presentations_pages:
        f.write("%s\n" % item)

### Visit each presentation and get video link

In [ ]:
extracted_links = []
heading_list = []
for presentation_link in list_presentations_pages:
    driver = webdriver.Chrome()
    driver.get(presentation_link)
    heading = driver.find_element_by_tag_name('h1').get_attribute('innerHTML')
    heading = heading.strip().replace("amp;", '_').replace(" ", "_")

    iframe_div = driver.find_element_by_class_name('slp__videoPlayer__content')
    iframe = iframe_div.find_element_by_tag_name('iframe')

    driver.switch_to.frame(iframe)

    html = driver.page_source
    soup = BeautifulSoup(html)
    script = soup.find_all('script')[4]
    res = re.search('"url":"(https://gcs-vimeo.akamaized.net.*?.mp4)"', script.string)
    print (res.group(1), heading)

    extracted_links.append(res.group(1))
    heading_list.append(heading)
    driver.quit()

#### Save names and links to disk

In [20]:
with open('all_urls_headings.txt', 'w') as f:
    for i in range(len(extracted_links)): 
        f.write("%s %s\n" % (heading_list[i], extracted_links[i]))

### Download all videos

In [ ]:
download_all_videos(extracted_links, heading_list)

  0%|          | 1/261 [04:05<17:44:06, 245.56s/it]